In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import PCAModel

In [2]:
city = "sf"

In [3]:
network = RoadNetwork()
network.load(f"../../osm_data/{city}")

In [6]:
from torch_geometric.data import Data
import torch_geometric.transforms as T

# create pyg dataset
data = network.generate_road_segment_pyg_dataset(include_coords=True, dataset=city)

model = PCAModel(data, emb_dim=4)
model.train()
# model.save_emb(path="../../model_states/pca/")

In [7]:
"""
Evaluate on road type classification
"""
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

X = model.emb # embedding for each node
# train simple classifier on 80% of data with cross validation
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# mask = ((y==11) | (y==10) | (y==9) | (y==4) | (y==1) | (y==2) | (y==12) | (y==7)) # remove uncommon tags
# X = X[~mask, :]
# y = y[~mask]
# print(np.unique(y, return_counts=True))

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)

print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_test dimension= ', y_test.shape)

lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
lm.fit(X_train, y_train)
print(metrics.classification_report(y_test, lm.predict(X_test)))

X_train dimension=  (21631, 4)
X_test dimension=  (5408, 4)
y_train dimension=  (21631,)
y_test dimension=  (5408,)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00        23
           3       0.00      0.00      0.00        35
           4       0.00      0.00      0.00       246
           5       0.00      0.00      0.00        16
           6       0.84      1.00      0.91      3574
           7       0.64      0.46      0.54       515
           8       0.00      0.00      0.00        19
           9       1.00      1.00      1.00       779
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00        53
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00       110

    accuracy                           0.85      5408
   macro avg      